In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install evaluate
!pip install sacrebleu

In [2]:
import tensorflow as tf

In [3]:
from datasets import load_dataset

C:\Users\PC\Anaconda3\envs\fkgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
en_ko = load_dataset("bongsoo/news_talk_en_ko")

Found cached dataset csv (C:/Users/PC/.cache/huggingface/datasets/bongsoo___csv/bongsoo--news_talk_en_ko-e7f00bc8f76f18d5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 50.17it/s]


In [5]:
en_ko

DatasetDict({
    train: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 1299999
    })
})

In [6]:
import pandas as pd
en_ko.set_format(type="pandas")
df = en_ko["train"][:]
df.head()

,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
0,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
1,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
2,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
3,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.
4,I can't even consider uninsured treatments.,보험 처리가 안 되는 비급여 시술은 엄두도 못 낸다.


In [7]:
example_0 = list(df.columns)
example_0
example_0_df = pd.DataFrame({col: [value] for col, value in zip(('en', 'ko'), example_0)})
df.columns = ('en', 'ko')
en_ko_df = pd.concat([example_0_df, df],).reset_index(drop=True)
en_ko_df.head()

,en,ko
0,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
1,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
2,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
3,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
4,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.


In [8]:
from datasets import Dataset
dataset = Dataset.from_pandas(en_ko_df)
dataset

Dataset({
    features: ['en', 'ko'],
    num_rows: 1300000
})

In [9]:
num_train = 1200000
num_valid = 90000
num_test = 10000
en_ko_df_train = en_ko_df.iloc[:num_train]
en_ko_df_valid = en_ko_df.iloc[num_train:num_train+num_valid]
en_ko_df_test = en_ko_df.iloc[-num_test:]
en_ko_df_train.to_csv("train.tsv", sep='\t', index=False)
en_ko_df_valid.to_csv("valid.tsv", sep='\t', index=False)
en_ko_df_test.to_csv("test.tsv", sep='\t', index=False)

In [10]:
data_files = {"train": "train.tsv", "valid": "valid.tsv", "test": "test.tsv"}
dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")
dataset

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 376.29it/s]


Dataset csv downloaded and prepared to C:/Users/PC/.cache/huggingface/datasets/csv/default-9b2c24984fac7a62/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 62.71it/s]


DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 1200000
    })
    valid: Dataset({
        features: ['en', 'ko'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['en', 'ko'],
        num_rows: 10000
    })
})

In [11]:
print(dataset['train']['en'][:3], dataset['train']['ko'][:3])
print(dataset['train'][:3]['en'], dataset['train'][:3]['ko'])


["Skinner's reward is mostly eye-watering.", 'Even some problems can be predicted.', 'Only God will exactly know why.'] ['스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.', '심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.', '오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.']
["Skinner's reward is mostly eye-watering.", 'Even some problems can be predicted.', 'Only God will exactly know why.'] ['스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.', '심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.', '오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.']


In [12]:
import numpy as np

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric


In [14]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')

In [15]:
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [16]:
import numpy as np
import torch
import multiprocessing
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070'

In [17]:
torch.cuda.is_available()


True

In [18]:
from torch.utils.data import Dataset

In [19]:

model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

"""## Tokenizer

먼저 모델 체크 포인트를 사용하여 KE-T5 모델이 학습할때 함께 사용한 토크나이저를 불러온다. 허깅페이스 트랜스포머스 라이브러리를 사용할 때 가장 핵심이 되며 익숙해지기 쉽지 않은 부분이 이 토크나이저라고 개인적으로 생각한다.
"""

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

"""토크나이저를 로딩할때 `sentencepiece`가 없다고 에러가 나면 위 제시한 라이브러리가 설치 안된 것이므로 설치하고 다시 시도한다.

토크나이저를 불러왔으니 현재 사용하는 데이터셋에서 샘플을 가져와 토크나이징해보는 것이 좋을 것이다. 학습 세트에서 10번 샘플을 가지고 실험해보자. 먼저 10번 샘플을 뿌려보고
"""


'토크나이저를 로딩할때 `sentencepiece`가 없다고 에러가 나면 위 제시한 라이브러리가 설치 안된 것이므로 설치하고 다시 시도한다.\n\n토크나이저를 불러왔으니 현재 사용하는 데이터셋에서 샘플을 가져와 토크나이징해보는 것이 좋을 것이다. 학습 세트에서 10번 샘플을 가지고 실험해보자. 먼저 10번 샘플을 뿌려보고\n'

In [20]:

dataset['train'][10]['en'], dataset['train'][10]['ko']



('Any academic achievement requires constant repetition.',
 '어떤 학문이든지 일정의 성취를 이루기 위해서는 끊임없는 반복이 필요하다.')

In [21]:
tokenized_sample_en = tokenizer(dataset['train'][10]['en'], 
                                max_length=max_token_length, 
                                padding=True, truncation=True)
tokenized_sample_en


{'input_ids': [13941, 10114, 25542, 9361, 20526, 742, 32268, 12520, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:

tokenized_sample_ko = tokenizer(dataset['train'][10]['ko'], 
                                max_length=max_token_length, 
                                padding=True, truncation=True)
tokenized_sample_ko


{'input_ids': [404, 12663, 15, 10775, 2334, 6, 15757, 21, 29819, 1736, 26778, 4342, 15, 1701, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:

tokenizer(dataset['train'][:3]['en'], 
          max_length=max_token_length, 
          padding=True, truncation=True)

"""미니배치에 있는 샘플의 최대길이메 맞춰서 패딩되는 모습을 확인할 수 있다. 실제로 어떻게 토큰화 되었는지 확인해보자."""

pd.DataFrame(
    [
        tokenized_sample_en['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_en['input_ids'])
    ], index=('ids', 'tokens')
)

pd.DataFrame(
    [
        tokenized_sample_ko['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_ko['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
ids,404,12663,15,10775,2334,6,15757,21,29819,1736,26778,4342,15,1701,3,1
tokens,▁어떤,▁학문,이,든지,▁일정,의,▁성취,를,▁이루기,▁위해서는,▁끊임없는,▁반복,이,▁필요하다,.,</s>


In [24]:
def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['en'],
                             text_target=examples['ko'], 
                             max_length=max_token_length, truncation=True)
    
    return model_inputs

In [25]:
examples= {'en':['sent1', 'sent2', ... , 'sent1000'], # 이건 문장 1000개짜리 리스트
           'ko':['sent1', 'sent2', ... , 'sent1000']}

In [26]:
NUM_CPU = multiprocessing.cpu_count() 
NUM_CPU

20

In [27]:

"""그리고 `remove_columns` 인자에 기존 특성 이름인 `en`, `ko`를 전달해서 기존 특성은 제거하게 한다. 이 특성이 있으면 이후 콜레이터가 샘플들을 미니 배치로 묶을 때 패딩처리를 못하게 된다. """

                              #  num_proc=NUM_CPU)
tokenized_datasets = dataset.map(
    convert_examples_to_features,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

In [28]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1200000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})

In [29]:
tokenized_datasets['train'][10]

{'input_ids': [13941, 10114, 25542, 9361, 20526, 742, 32268, 12520, 3, 1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [404,
  12663,
  15,
  10775,
  2334,
  6,
  15757,
  21,
  29819,
  1736,
  26778,
  4342,
  15,
  1701,
  3,
  1]}

In [30]:
print( '원 데이터    :', dataset['train'][10]['en'] )
print( '처리 후 데이터:', tokenized_datasets['train'][10]['input_ids'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['input_ids']) )

print('\n')
print( '원 데이터    :', dataset['train'][10]['ko'] )
print( '처리 후 데이터:', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['labels']) )
print( '토큰화       :', tokenized_datasets['train'][10]['labels'] )

원 데이터    : Any academic achievement requires constant repetition.
처리 후 데이터: [13941, 10114, 25542, 9361, 20526, 742, 32268, 12520, 3, 1]
토큰화       : ['▁Any', '▁academic', '▁achievement', '▁requires', '▁constant', '▁re', 'pet', 'ition', '.', '</s>']


원 데이터    : 어떤 학문이든지 일정의 성취를 이루기 위해서는 끊임없는 반복이 필요하다.
처리 후 데이터: ['▁어떤', '▁학문', '이', '든지', '▁일정', '의', '▁성취', '를', '▁이루기', '▁위해서는', '▁끊임없는', '▁반복', '이', '▁필요하다', '.', '</s>']
토큰화       : [404, 12663, 15, 10775, 2334, 6, 15757, 21, 29819, 1736, 26778, 4342, 15, 1701, 3, 1]


In [31]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [32]:
encoder_inputs = tokenizer(
    ["Studies have been shown that owning a dog is good for you"], 
    return_tensors="pt"
)['input_ids'].to(device)

decoder_targets = tokenizer(
    ["개를 키우는 것이 건강에 좋다는 연구 결과가 있습니다."], 
    return_tensors="pt"
)['input_ids'].to(device)

In [33]:
print( encoder_inputs )
print( decoder_targets )

tensor([[24611,    84,   166,  8135,    38,   847,    91,    16,  8146,    43,
           667,    40,   106,     1]], device='cuda:0')
tensor([[15833, 12236,   179, 16120, 28117,  1007,  3883,   327,     3,     1]],
       device='cuda:0')


In [34]:
decoder_inputs = model._shift_right(decoder_targets)

In [35]:
pd.DataFrame(
    [
        tokenizer.convert_ids_to_tokens(decoder_targets[0]),
        tokenizer.convert_ids_to_tokens(decoder_inputs[0])
    ],
    index=('decoder target', 'decoder input')
)

,0,1,2,3,4,5,6,7,8,9
decoder target,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.,</s>
decoder input,<pad>,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.


In [36]:
outputs = model(input_ids=encoder_inputs, 
                decoder_input_ids=decoder_inputs, 
                labels=decoder_targets)

In [37]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

In [38]:
outputs.loss

tensor(87.8281, device='cuda:0', grad_fn=<NllLossBackward0>)

In [39]:
outputs['encoder_last_hidden_state'].shape

torch.Size([1, 14, 768])

In [40]:
outputs['logits'].shape

torch.Size([1, 10, 64128])

In [41]:
tokenizer.convert_ids_to_tokens( torch.argmax(outputs['logits'][0], axis=1).cpu().numpy() )

['큐브', '큐브', '▁비일비재', '▁비일비재', '▁베네', '▁비일비재', '▁베네', '▁베네', '큐브', '큐브']

In [42]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
tokenized_datasets["train"][1:3]

{'input_ids': [[4014, 322, 3170, 147, 67, 23274, 3, 1],
  [11783, 4412, 96, 6556, 709, 1632, 3, 1]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]],
 'labels': [[6842, 404, 951, 5767, 15387, 27, 831, 800, 4378, 15, 1587, 3, 1],
  [9881, 18590, 3837, 70, 4341, 1086, 677, 35, 426, 2255, 3, 1]]}

In [43]:
[tokenized_datasets["train"][i] for i in range(1, 3)]

[{'input_ids': [4014, 322, 3170, 147, 67, 23274, 3, 1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [6842, 404, 951, 5767, 15387, 27, 831, 800, 4378, 15, 1587, 3, 1]},
 {'input_ids': [11783, 4412, 96, 6556, 709, 1632, 3, 1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [9881, 18590, 3837, 70, 4341, 1086, 677, 35, 426, 2255, 3, 1]}]

In [44]:
batch = data_collator(
    [tokenized_datasets["train"][i] for i in range(1, 3)]
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [45]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [46]:
batch

{'input_ids': tensor([[ 4014,   322,  3170,   147,    67, 23274,     3,     1],
        [11783,  4412,    96,  6556,   709,  1632,     3,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ 6842,   404,   951,  5767, 15387,    27,   831,   800,  4378,    15,
          1587,     3,     1],
        [ 9881, 18590,  3837,    70,  4341,  1086,   677,    35,   426,  2255,
             3,     1,  -100]]), 'decoder_input_ids': tensor([[    0,  6842,   404,   951,  5767, 15387,    27,   831,   800,  4378,
            15,  1587,     3],
        [    0,  9881, 18590,  3837,    70,  4341,  1086,   677,    35,   426,
          2255,     3,     1]])}

In [47]:
import evaluate

metric = evaluate.load("sacrebleu")

In [48]:
predictions = [
    "저는 딥러닝을 좋아해요.",
    "요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다."
]

references = [
    ["저는 딥러닝을 좋아해요.", "나는 딥러닝을 사랑해요."],
    ["요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다.",
     "최근에는 딥러닝 프레임워크가 잘 개발되어 있기 때문에 다른 사람의 도움 없이도 기계 번역 시스템을 개발할 수 있습니다."]
]
metric.compute(predictions=predictions, references=references)

{'score': 100.00000000000004,
 'counts': [21, 19, 17, 15],
 'totals': [21, 19, 17, 15],
 'precisions': [100.0, 100.0, 100.0, 100.0],
 'bp': 1.0,
 'sys_len': 21,
 'ref_len': 21}

In [49]:
predictions = [
    "저는 딥러닝을 좋아해요.",
    "딥러닝 프레임워크가 잘 개발되었기 때문에 요즘은 누군가의 도움 없이 기계번역 시스템을 구축할 수 있다."
]

references = [
    ["저는 딥러닝을 좋아해요.", "나는 딥러닝을 사랑해요."],
    ["요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다.",
     "최근에는 딥러닝 프레임워크가 잘 개발되어 있기 때문에 다른 사람의 도움 없이도 기계 번역 시스템을 개발할 수 있습니다."]
]
metric.compute(predictions=predictions, references=references)

{'score': 25.28116160010779,
 'counts': [14, 7, 4, 1],
 'totals': [19, 17, 15, 13],
 'precisions': [73.6842105263158,
  41.1764705882353,
  26.666666666666668,
  7.6923076923076925],
 'bp': 0.9000876262522591,
 'sys_len': 19,
 'ref_len': 21}

In [50]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    
    return result

In [53]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.28.0
Uninstalling transformers-4.28.0:
  Successfully uninstalled transformers-4.28.0
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0


  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)


In [51]:
pip install transformers==4.28.0

Note: you may need to restart the kernel to use updated packages.


In [52]:

import gc
torch.cuda.empty_cache()
gc.collect()

769

In [54]:
training_args = Seq2SeqTrainingArguments(
    output_dir="chkpt",
    learning_rate=0.0005,
    weight_decay=0.01,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=128,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_strategy="no",
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    report_to="none" # Wandb 로그 끄기
)

In [55]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./results2")

In [91]:
model_dir = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu();

In [92]:
model_dir = "./results2"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu();

In [93]:
model.cpu();

In [94]:
input_text = [
    "You will face many defeats in life, but never let yourself be defeated.",
    "The greatest glory in living lies not in never falling, but in rising every time we fall",
    "Nice to meet you,brother"
]

In [95]:
inputs = tokenizer(input_text, return_tensors="pt", 
                   padding=True, max_length=max_token_length)

In [96]:
inputs

{'input_ids': tensor([[  988,    96,  2267,   494, 10503,     8,    20,   899,     4,   134,
          1249,  2691,  8267,    67, 20362,     3,     1,     0,     0],
        [   55, 12949, 45992,    20,  3144, 18635,   112,    20,  1249, 13223,
             4,   134,    20,  9413,  1073,   248,   146,  3653,     1],
        [51817,    10,  3391,   106,     4, 24520, 15422,     1,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [73]:
koreans = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

koreans.shape

torch.Size([3, 19])

In [79]:
[ 
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(korean)) for korean in koreans
]

['<pad> 인생에서 많은 패배를 당하겠지만, 절대 패배하지 마세요.</s><pad><pad><pad><pad><pad><pad>',
 '<pad> 살면서 가장 큰 영광은 결코 떨어지지 않는 것이 아니라 우리가 떨어질 때마다 오르는 것이에요.</s>',
 '<pad> 당신과 만나 뵙게 되어 기<unk>니다, 오빠.</s><pad><pad><pad><pad><pad>']

In [80]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

In [81]:
test_dataloader_iter = iter(test_dataloader)

In [82]:
test_batch = next(test_dataloader_iter)

In [83]:
test_batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [84]:
test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }

In [85]:
koreans = model.generate(
    **test_input,
    max_length=max_token_length,
    num_beams=5,
)

In [86]:
labels =  np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)

In [87]:
eng_sents = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True)[10:20]

In [88]:
references = tokenizer.batch_decode(labels, skip_special_tokens=True)[10:20]

In [89]:
preds = tokenizer.batch_decode( koreans, skip_special_tokens=True )[10:20]

In [90]:
for s in zip(eng_sents, references, preds):
    print('English   :', s[0])
    print('Reference :', s[1])
    print('Translated:', s[2])
    print('\n')

English   : Yes, I'll see you at the parking lot at 3 p.m.
Reference : 네, 오후 3시에 주차장에서 뵙죠.
Translated: 네, 오후 3시에 주차장에서 뵙겠습니다.


English   : I'm happy to see Jessica Huh take over my role.
Reference : Jessica Huh가 제 역할을 인계받게 되어서 저는 기니다.
Translated: 제시카 허가 제 역할을 맡게 되어 기니다.


English   : I agree with you that she is qualified for the position.
Reference : 저도 부장님 의견대로 그녀가 이 자리의 적임자라고 생각합니다.
Translated: 저는 그녀가 그 자리에 자격이 있다는 것에 동의합니다.


English   : Nick, I was told that your department will be divided into two.
Reference : Nick, 당신 부서가 둘로 나뉜다면서요?
Translated: 닉, 당신의 부서가 두 개로 나뉘게 될 거라고 들었습니다.


English   : Yes, all staff involved in online advertising will have their own office space located on the 2nd floor.
Reference : 네, 다음 달 초부터 온라인 광고와 관련된 직원들은 2층에 위치한 개별 사무실 공간을 사용한다고 합니다.
Translated: 네, 온라인 광고에 관여하는 모든 직원이 2층에 각자의 오피스 공간을 가질 것입니다.


English   : What happens to the remaining staff?
Reference : 남은 직원들은 어떻게 되나요?
Translated: 남은 스태프에게 무슨 일이 생겼나요?


English   : The remaining staff will stay i